In [5]:
%load_ext autoreload
%autoreload 2

import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.layers import Dense, Flatten, Input, Reshape
from keras.models import Sequential, Model
from PIL import Image
from pprint import pprint

from keras_preprocessing.image import DataFrameIterator, ImageDataGenerator


pixel_val = 1
np_filenames, img_filenames, np_arrays = [], [], []
for i in range(10):
    np_filename = '/tmp/{}.npy'.format(i)
    img_filename = '/tmp/{}.jpg'.format(i)
    array = pixel_val * np.random.randint(0, 255, size=(2, 2)).astype('float32')
    np.save(np_filename, array[..., None])
    plt.imsave(img_filename, array)
    
    np_filenames.append(np_filename)
    img_filenames.append(img_filename)
    
df = pd.DataFrame({'img_path': img_filenames, 'img_np_path': np_filenames})
df['regression'] = np.random.randint(1, 10, size=len(df))
df['binary'] = random.sample(['dog', 'cat'] * 30, len(df))
df['multi_class'] = random.sample(['dog', 'cat', 'horse'] * 30, len(df))
df['multi_label'] = random.sample(['dog', 'cat', ['dog'], ['cat'], ['cat', 'dog'], ['dog', 'cat']] * 10, len(df))
df['img_path_extra'] = img_filenames
df['regression_extra'] = np.random.randint(10, 100, size=len(df))

df.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,img_path,img_np_path,regression,binary,multi_class,multi_label,img_path_extra,regression_extra
0,/tmp/0.jpg,/tmp/0.npy,7,dog,cat,"[dog, cat]",/tmp/0.jpg,81
1,/tmp/1.jpg,/tmp/1.npy,6,cat,cat,"[dog, cat]",/tmp/1.jpg,89
2,/tmp/2.jpg,/tmp/2.npy,6,dog,cat,dog,/tmp/2.jpg,40
3,/tmp/3.jpg,/tmp/3.npy,2,cat,dog,"[cat, dog]",/tmp/3.jpg,42
4,/tmp/4.jpg,/tmp/4.npy,6,dog,dog,[cat],/tmp/4.jpg,76


### Regression & regression

In [6]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['regression', 'regression_extra'],
    image_size=(2, 2),
    batch_size=2,
    color_mode='grayscale'
)

print('\nInputs:')
pprint(df_iter.inputs)
print('\nOutputs:')
pprint(df_iter.outputs)

_input = Input(shape=(2, 2, 1))
x = Flatten()(_input)
out_1 = Dense(1)(x)
out_2 = Dense(1)(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', 'mse')
model.fit_generator(df_iter)


Inputs:
[{'column': 'img_path',
  'values': ['/tmp/0.jpg',
             '/tmp/1.jpg',
             '/tmp/2.jpg',
             '/tmp/3.jpg',
             '/tmp/4.jpg',
             '/tmp/5.jpg',
             '/tmp/6.jpg',
             '/tmp/7.jpg',
             '/tmp/8.jpg',
             '/tmp/9.jpg']}]

Outputs:
[{'column': 'regression',
  'mode': None,
  'values': [7, 6, 6, 2, 6, 8, 3, 9, 4, 8]},
 {'column': 'regression_extra',
  'mode': None,
  'values': [81, 89, 40, 42, 76, 57, 54, 69, 28, 59]}]
Epoch 1/1
5/5 [==============================] - 0s 21ms/step - loss: 46034.2750 - dense_7_loss: 19555.9961 - dense_8_loss: 26478.2754


### Regression & binary classification

In [7]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['regression', 'binary'],
    output_modes={'binary': 'sparse'},
    image_size=(2, 2),
    batch_size=2,
    color_mode='grayscale'
)

print('\nInputs:')
pprint(df_iter.inputs)
print('\nOutputs:')
pprint(df_iter.outputs)

_input = Input(shape=(2, 2, 1))
x = Flatten()(_input)
out_1 = Dense(1)(x)
out_2 = Dense(1, activation='sigmoid')(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', loss=['mse', 'binary_crossentropy'])
model.fit_generator(df_iter)


Inputs:
[{'column': 'img_path',
  'values': ['/tmp/0.jpg',
             '/tmp/1.jpg',
             '/tmp/2.jpg',
             '/tmp/3.jpg',
             '/tmp/4.jpg',
             '/tmp/5.jpg',
             '/tmp/6.jpg',
             '/tmp/7.jpg',
             '/tmp/8.jpg',
             '/tmp/9.jpg']}]

Outputs:
[{'column': 'regression',
  'mode': None,
  'values': [7, 6, 6, 2, 6, 8, 3, 9, 4, 8]},
 {'column': 'binary',
  'index_to_class': {0: 'cat', 1: 'dog'},
  'mode': 'sparse',
  'values': [1, 0, 1, 0, 1, 1, 1, 1, 1, 1]}]
Epoch 1/1
5/5 [==============================] - 0s 29ms/step - loss: 14716.0994 - dense_9_loss: 14659.1436 - dense_10_loss: 56.9555


### Regression & multi-class classification

In [8]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['regression', 'multi_class'],
    output_modes={'multi_class': 'categorical'},
    image_size=(2, 2),
    batch_size=2,
    color_mode='grayscale'
)

print('\nInputs:')
pprint(df_iter.inputs)
print('\nOutputs:')
pprint(df_iter.outputs)

_input = Input(shape=(2, 2, 1))
x = Flatten()(_input)
out_1 = Dense(1)(x)
out_2 = Dense(3, activation='softmax')(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', loss=['mse', 'categorical_crossentropy'])
model.fit_generator(df_iter)


Inputs:
[{'column': 'img_path',
  'values': ['/tmp/0.jpg',
             '/tmp/1.jpg',
             '/tmp/2.jpg',
             '/tmp/3.jpg',
             '/tmp/4.jpg',
             '/tmp/5.jpg',
             '/tmp/6.jpg',
             '/tmp/7.jpg',
             '/tmp/8.jpg',
             '/tmp/9.jpg']}]

Outputs:
[{'column': 'regression',
  'mode': None,
  'values': [7, 6, 6, 2, 6, 8, 3, 9, 4, 8]},
 {'column': 'multi_class',
  'index_to_class': {0: 'cat', 1: 'dog', 2: 'horse'},
  'mode': 'categorical',
  'values': [0, 0, 0, 1, 1, 0, 0, 1, 2, 0]}]
Epoch 1/1
5/5 [==============================] - 0s 22ms/step - loss: 19225.6564 - dense_11_loss: 19144.3613 - dense_12_loss: 81.2957


### Multi-class & multi-label classification

In [9]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['multi_class', 'multi_label'],
    output_modes={'multi_class': 'categorical', 'multi_label': 'categorical'},
    image_size=(2, 2),
    batch_size=2,
    color_mode='grayscale'
)

print('\nInputs:')
pprint(df_iter.inputs)
print('\nOutputs:')
pprint(df_iter.outputs)

_input = Input(shape=(2, 2, 1))
x = Flatten()(_input)
out_1 = Dense(3, activation='softmax')(x)
out_2 = Dense(2, activation='sigmoid')(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', loss=['categorical_crossentropy', 'binary_crossentropy'])
model.fit_generator(df_iter)


Inputs:
[{'column': 'img_path',
  'values': ['/tmp/0.jpg',
             '/tmp/1.jpg',
             '/tmp/2.jpg',
             '/tmp/3.jpg',
             '/tmp/4.jpg',
             '/tmp/5.jpg',
             '/tmp/6.jpg',
             '/tmp/7.jpg',
             '/tmp/8.jpg',
             '/tmp/9.jpg']}]

Outputs:
[{'column': 'multi_class',
  'index_to_class': {0: 'cat', 1: 'dog', 2: 'horse'},
  'mode': 'categorical',
  'values': [0, 0, 0, 1, 1, 0, 0, 1, 2, 0]},
 {'column': 'multi_label',
  'index_to_class': {0: 'cat', 1: 'dog'},
  'mode': 'categorical',
  'values': [[1, 0], [1, 0], 1, [0, 1], [0], 0, [0, 1], [1, 0], 1, 1]}]
Epoch 1/1
5/5 [==============================] - 0s 28ms/step - loss: 189.8079 - dense_13_loss: 108.1421 - dense_14_loss: 81.6658
